In [24]:
import numpy as np
import pandas as pd
import os
from pprint import pprint
from IPython.display import display

import tensorflow as tf
from tensorflow import keras
from tensorflow.io import FixedLenFeature

import matplotlib.pyplot as plt

In [34]:
class CFG:
    debug=True if "get_ipython" in globals() else False

    dataset_dir="../input/ranzcr-clip-catheter-line-classification/"
    target_size=(256,256)
    batch_size=4 if debug else 128
    epochs=1 if debug else 20

    target_cols=["ETT - Abnormal","ETT - Borderline","ETT - Normal","NGT - Abnormal","NGT - Borderline","NGT - Incompletely Imaged",
        "NGT - Normal","CVC - Abnormal","CVC - Borderline","CVC - Normal","Swan Ganz Catheter Present"]

print(CFG.debug)

True


In [3]:
feature_description={
    "CVC - Abnormal":FixedLenFeature([],tf.int64),
    "CVC - Borderline":FixedLenFeature([],tf.int64),
    "CVC - Normal":FixedLenFeature([],tf.int64),
    "ETT - Abnormal":FixedLenFeature([],tf.int64),
    "ETT - Borderline":FixedLenFeature([],tf.int64),
    "ETT - Normal":FixedLenFeature([],tf.int64),
    "NGT - Abnormal":FixedLenFeature([],tf.int64),
    "NGT - Borderline":FixedLenFeature([],tf.int64),
    "NGT - Incompletely Imaged":FixedLenFeature([],tf.int64),
    "NGT - Normal":FixedLenFeature([],tf.int64),
    "Swan Ganz Catheter Present":FixedLenFeature([],tf.int64),
    "StudyInstanceUID":FixedLenFeature([],tf.string),
    "image":FixedLenFeature([],tf.string),
}

In [22]:
def parse_batch_example(example):
    return tf.io.parse_example(example,feature_description)

def preprocessing(parsed_record):
    image=tf.io.decode_jpeg(parsed_record["image"],channels=3) # decode_imageではresizeでエラーが出る
    image=tf.cast(image,tf.float32)
    image=tf.image.resize(image,CFG.target_size)
    image/=255.0

    label=[]
    for col_name in CFG.target_cols:
        label.append(parsed_record[col_name])
    
    return image,label


def build_dataset(repeat=False):
    AUTO=tf.data.experimental.AUTOTUNE

    os.makedirs("./cache/",exist_ok=True)

    filenames=[f"{CFG.dataset_dir}train_tfrecords/{str(n).zfill(2)}-1881.tfrec" for n in range(16)]
    dset=tf.data.TFRecordDataset(filenames)
    dset=dset.map(parse_batch_example,num_parallel_calls=AUTO)
    dset=dset.map(preprocessing,num_parallel_calls=AUTO) # time_consuming_mapping
    dset=dset.cache("./cache/")
    dset=dset.repeat() if repeat else dset
    dset=dset.batch(CFG.batch_size).prefetch(AUTO)

    return dset

In [25]:
def image_show(image,title=None):
    fig,axes=plt.subplots(1,1,figsize=(5,5))
    axes.imshow(image)
    axes.axis(False)
    if title:
        axes.set_title(parsed_record["StudyInstanceUID"].numpy().decode())


train_dataset=build_dataset()

In [32]:
model=keras.Sequential([
    keras.layers.Flatten(input_shape=(256,256,3)),
    keras.layers.Dense(11,activation="sigmoid")
])

model.compile(optimizer="adam",loss="binary_crossentropy",metrics=[keras.metrics.AUC(multi_label=True)])

In [33]:
model.fit(train_dataset,epochs=CFG.epochs)

   1739/Unknown - 82s 47ms/step - loss: 4.3811 - auc_1: 0.5534

KeyboardInterrupt: 